In [13]:
import math
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import itertools

from ema_workbench import save_results, load_results
from ema_workbench.analysis import parcoords
from ema_workbench.analysis import clusterer, plotting, Density

#%matplotlib inline
clr_palette = ([sns.color_palette("YlGn", 15)[10],sns.cubehelix_palette(8)[6]])

pydice_folder = "C:/Users/ivart/OneDrive/Bureaublad/Afstuderen/WRR/PyRICE Ivar Tjallingii/PyRICE2020/6_Uncertainty Analysis/server/model_server"
print(pydice_folder)
sys.path.append(pydice_folder)

from ema_workbench import (perform_experiments, Model, Policy, Scenario, ReplicatorModel, RealParameter, IntegerParameter, ScalarOutcome, ArrayOutcome, 
                           Constant, ema_logging, SequentialEvaluator, MultiprocessingEvaluator, IpyparallelEvaluator)

from PyRICE_V9_short_term_uncertainty import PyRICE

from matplotlib import pyplot

from model_outcomes_uncertainty_v2 import get_all_model_outcomes_uncertainty_search

C:/Users/ivart/OneDrive/Bureaublad/Afstuderen/WRR/PyRICE Ivar Tjallingii/PyRICE2020/6_Uncertainty Analysis/server/model_server


## 1. Obtain output frames

In [14]:
timepoints_to_save = np.arange(2005,2105+10,10)
objectives_list_name = ['Intertemporal utility GINI','Intertemporal impact GINI','Total Aggregated Utility','Regions below treshold']

objectives_list_timeseries_name = ['Damages ','Utility ',
            'Lowest income per capita ','Highest climate impact per capita ',
            'Distance to treshold ','Population under treshold ',
            'Intratemporal utility GINI ','Intratemporal impact GINI ',
            'Atmospheric Temperature ', 'Industrial Emission ', 'Total Output ']

supplementary_list_timeseries_name = ['CPC ','Population ']
supplementary_list_quintile_name = ['CPC pre damage ','CPC post damage ']

outcomes_names_time_series = []
for name in objectives_list_timeseries_name:
    for year in timepoints_to_save:
        name_year = name + str(year)
        outcomes_names_time_series.append(name_year)

outcomes_name_single = []
outcomes_name_multiple= []
for name in objectives_list_name:
    if name == "Regions below treshold":
        outcomes_name_multiple.append(name)
    else:
        outcomes_name_single.append(name)

outcome_name_supplementary =[]
for name in supplementary_list_timeseries_name:
    for year in timepoints_to_save:
        name_year = name + str(year)
        outcome_name_supplementary.append(name_year)
        
outcome_name_supplementary_quintile=[]
for name in supplementary_list_quintile_name:
    for year in timepoints_to_save:
        name_year = name + str(year)
        outcome_name_supplementary_quintile.append(name_year)
        

### Format for dataframes (index and column names)

In [15]:
RICE_POP_gr = pd.read_excel("RICE_2010_base_000.xlsm", sheet_name="Pop_gr")

regions_list = ["US", "OECD-Europe","Japan","Russia","Non-Russia Eurasia","China","India","Middle East","Africa",
    "Latin America","OHI","Other non-OECD Asia"]

"""
####################### Population PARAMETERS and set up dataframe format #######################
"""

#get population growth rates for each region
a=[]
for i in range(31):  
    if i == 0:
        a.append("region")
    k = 2005 + 10 * i
    k = str(k)
    a.append(k)    

region_pop_gr = RICE_POP_gr.iloc[10:22,3:35]
region_pop_gr.columns =  a
region_pop_gr = region_pop_gr.set_index('region') 

#### Get all policies - 30 policies including Nordhaus

In [16]:
all_policies = pd.read_csv("example_policys_principles.csv",index_col=0)

principles_list =["utilitarian","prioritarian","egalitarian","sufficitarian","nordhaus"]

total_policy_list = []

for principle in principles_list:
    policies = all_policies[all_policies['principle']==principle]
    policies = policies.dropna(axis='columns')
    policies = policies.iloc[:,:-1]
    policy_list_principle = []

    #get list of policies as input for uncertainty sampling
    for i in range(0,len(policies)):
        policy_dict = policies.iloc[i].to_dict()
        policy_list_principle.append(Policy(policies.index[i], **policy_dict)) 
    total_policy_list.append(policy_list_principle)
total_policy_list[4]

[Policy({'sr': 0.248, 'irstp': 0.015, 'miu_period': 13.0})]

In [5]:
from load_result import load_result

### load in data one by one to avoid overloading memory

In [12]:
experiments_nord.columns

Index(['fdamage', 'fosslim', 'scenario_cback',
       'scenario_elasticity_of_damages', 'scenario_limmiu', 'scenario_pop_gdp',
       'scenario_sigma', 't2xco2_dist', 't2xco2_index', 'scenario', 'policy',
       'model', 'sr', 'irstp', 'miu_period'],
      dtype='object')

In [17]:
result_nordhaus = load_result('results_uncertainty_analsysis_short_term_nordhaus_runs_35000.tar.gz')
experiments_nord, outcomes_nord = result_nordhaus

In [18]:
result_util = load_result('results_uncertainty_analsysis_short_term_utilitarian_runs_35000.tar.gz')
experiments_util, outcomes_util = result_util

In [19]:
result_ega_4_8 = load_result('results_uncertainty_analsysis_short_term_policies_4-8egalitarian_runs_35000.tar.gz')
experiments_ega_4_8, outcomes_ega_4_8 = result_ega_4_8

In [20]:
result_ega_8_8 = load_result('results_uncertainty_analsysis_short_term_policies_8-8egalitarian_runs_35000.tar.gz')
experiments_ega_8_8, outcomes_ega_8_8 = result_ega_8_8

In [21]:
results_suf_4_8 = load_result('results_uncertainty_analsysis_short_term_policies_4-8sufficitarian_runs_35000.tar.gz')
experiments_suf_4_8, outcomes_suf_4_8 = results_suf_4_8

In [22]:
results_suf_8_8 = load_result('results_uncertainty_analsysis_short_term_policies_8-8sufficitarian_runs_35000.tar.gz')
experiments_suf_8_8, outcomes_suf_8_8 = results_suf_8_8

In [23]:
results_prio_4_8 = load_result('results_uncertainty_analsysis_short_term_policies_4-8prioritarian_runs_35000.tar.gz')
experiments_prio_4_8, outcomes_prio_4_8 = results_prio_4_8

In [24]:
results_prio_8_8 = load_result('results_uncertainty_analsysis_short_term_policies_8-8prioritarian_runs_35000.tar.gz')
experiments_prio_8_8, outcomes_prio_8_8 = results_prio_8_8

## store single outputs in a csv file

In [25]:
outcomes_prio_8_8

{'Damages 2005': array([0.08727239, 0.22092299, 0.00159841, ..., 0.08727239, 0.22092299,
        0.00159841]),
 'Damages 2015': array([0.25856475, 0.17141358, 0.00090247, ..., 0.25856475, 0.17217682,
        0.00090247]),
 'Damages 2025': array([0.57498476, 0.        , 0.00105259, ..., 0.55256172, 0.42321644,
        0.00249631]),
 'Damages 2035': array([1.02128825, 0.14841303, 0.00156987, ..., 0.95851689, 0.91228045,
        0.00748345]),
 'Damages 2045': array([1.51242279, 0.        , 0.00213656, ..., 1.3821105 , 1.63430265,
        0.01929806]),
 'Damages 2055': array([1.99296426, 0.36134044, 0.00272649, ..., 1.74830273, 2.50475821,
        0.03809753]),
 'Damages 2065': array([2.43267604e+00, 0.00000000e+00, 3.24380326e-03, ...,
        2.00006054e+00, 3.36636015e+00, 5.55880451e-02]),
 'Damages 2075': array([2.83207177e+00, 5.67141852e-01, 3.64188652e-03, ...,
        2.14132877e+00, 4.11222188e+00, 6.44322611e-02]),
 'Damages 2085': array([3.20123068e+00, 0.00000000e+00, 3.913424

In [26]:
experiments_total= pd.DataFrame()

experiments_util["principle"] = "Utilitarian"
experiments_ega_4_8["principle"] = "Egalitarian"
experiments_ega_8_8["principle"] = "Egalitarian"
experiments_suf_4_8["principle"] = "Sufficitarian"
experiments_suf_8_8["principle"] = "Sufficitarian"
experiments_prio_4_8["principle"] = "Prioritarian"
experiments_prio_8_8["principle"] = "Prioritarian"
experiments_nord["principle"] = "Nordhaus policy"

outcomes_names_main = outcomes_names_time_series+outcomes_name_single

for keys in outcomes_names_main:
    experiments_util[keys] = outcomes_util[keys]
    experiments_ega_4_8[keys] = outcomes_ega_4_8[keys] 
    experiments_ega_8_8[keys] = outcomes_ega_8_8[keys] 
    experiments_suf_4_8[keys] = outcomes_suf_4_8[keys] 
    experiments_suf_8_8[keys] = outcomes_suf_8_8[keys] 
    experiments_prio_4_8[keys] = outcomes_prio_4_8[keys] 
    experiments_prio_8_8[keys] = outcomes_prio_8_8[keys] 
    experiments_nord[keys] = outcomes_nord[keys]

experiments_total = pd.concat([experiments_ega_4_8,experiments_ega_8_8,experiments_suf_4_8,
                               experiments_suf_8_8,experiments_prio_4_8,experiments_prio_8_8,
                               experiments_util,experiments_nord],axis=0)

In [27]:
experiments_total[experiments_total['policy']=="nordhaus"]

,fdamage,fosslim,scenario_cback,scenario_elasticity_of_damages,scenario_limmiu,scenario_pop_gdp,scenario_sigma,t2xco2_dist,t2xco2_index,scenario,...,Total Output 2095,Total Output 2105,Intertemporal utility GINI,Intertemporal impact GINI,Total Aggregated Utility,sufficitarian growth factor,sufficitarian_discounting,treshold level,growth_factor_prio,prioritarian_discounting
0,2.0,7099.419397,0.0,0.0,0.0,3.0,0.0,2.0,682.0,0,...,245.609708,254.333533,0.706125,0.862600,700.216635,NaN,NaN,NaN,NaN,NaN
1,0.0,13291.503245,1.0,2.0,1.0,2.0,0.0,1.0,884.0,1,...,362.921005,376.431931,0.724011,0.820365,1206.885141,NaN,NaN,NaN,NaN,NaN
2,0.0,5911.972098,1.0,2.0,1.0,3.0,1.0,2.0,709.0,2,...,241.286407,249.759727,0.705165,0.774110,690.514037,NaN,NaN,NaN,NaN,NaN
3,2.0,6148.027602,1.0,2.0,1.0,1.0,2.0,1.0,395.0,3,...,375.118010,380.903464,0.733571,0.885399,1222.801792,NaN,NaN,NaN,NaN,NaN
4,2.0,7229.580767,1.0,2.0,0.0,1.0,1.0,0.0,31.0,4,...,379.800070,387.339095,0.734382,0.883887,1224.669589,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34995,2.0,5667.799981,1.0,2.0,1.0,1.0,1.0,0.0,779.0,34995,...,377.162300,384.150146,0.733883,0.878312,1222.930416,NaN,NaN,NaN,NaN,NaN
34996,2.0,6660.323400,1.0,0.0,1.0,2.0,0.0,0.0,169.0,34996,...,369.939473,383.798675,0.724897,0.883872,1219.159165,NaN,NaN,NaN,NaN,NaN
34997,0.0,8769.036526,1.0,2.0,0.0,1.0,0.0,1.0,252.0,34997,...,377.648777,385.728392,0.734371,0.799734,1218.541783,NaN,NaN,NaN,NaN,NaN
34998,1.0,8134.068795,1.0,2.0,0.0,2.0,0.0,2.0,166.0,34998,...,361.404742,374.861064,0.723840,0.832298,1205.609514,NaN,NaN,NaN,NaN,NaN


In [28]:
experiments_total['principle']

0            Egalitarian
1            Egalitarian
2            Egalitarian
3            Egalitarian
4            Egalitarian
              ...       
34995    Nordhaus policy
34996    Nordhaus policy
34997    Nordhaus policy
34998    Nordhaus policy
34999    Nordhaus policy
Name: principle, Length: 1050000, dtype: object

In [29]:
experiments_total.to_csv("experiments_short_term_uncertainty_total.csv")

### get per region outputs and save in CSV

In [ ]:
len(outcomes_util['Damages 2005'])

In [ ]:
pop_keys = outcome_name_supplementary[0:11]
pop_keys

In [ ]:
#save CPC as csv
pop_keys = outcome_name_supplementary[0:11]

tuples = ()
index = pd.MultiIndex.from_tuples(tuples,names=['principle', 'year'])
CPC = pd.DataFrame(columns= region_pop_gr.index,index = index)

principles = ['egalitarian','egalitarian','sufficitarian','sufficitarian','prioritarian','prioritarian',
              'utilitarian']
outcomes = [outcomes_ega_4_8,outcomes_ega_8_8,outcomes_suf_4_8,
            outcomes_suf_8_8,outcomes_prio_4_8,outcomes_prio_8_8,
            outcomes_util]

index_year = 0
principle_index =0 

for outcome_principle in outcomes:
    for year in pop_keys:
        length = (len(outcome_principle[year]))
        if principles[principle_index] =="utilitarian":
            tuples = [(principles[principle_index],2005+index_year),]*(175*10**3)
        else:
            tuples = [(principles[principle_index],2005+index_year),]*14*10**4
        index_1 = pd.MultiIndex.from_tuples(tuples, names=['principle', 'year'])
        year_df = pd.DataFrame(data =outcome_principle[year], columns= region_pop_gr.index,index=index_1)

        index_year = index_year +10
        CPC = pd.concat([CPC,year_df],axis=0)
    principle_index = principle_index + 1


In [ ]:
CPC

In [ ]:
CPC.to_csv("short_term_CPC_per_time_period.csv")

In [ ]:
#save CPC as csv
pop_keys = outcome_name_supplementary[11:22]

tuples = ()
index = pd.MultiIndex.from_tuples(tuples,names=['principle', 'year'])
POP_total = pd.DataFrame(columns= region_pop_gr.index,index = index)

principles = ['egalitarian','egalitarian','sufficitarian','sufficitarian','prioritarian','prioritarian',
              'utilitarian']
outcomes = [outcomes_ega_4_8,outcomes_ega_8_8,outcomes_suf_4_8,
            outcomes_suf_8_8,outcomes_prio_4_8,outcomes_prio_8_8,
            outcomes_util]

index_year = 0
principle_index =0 

for outcome_principle in outcomes:
    for year in pop_keys:
        length = (len(outcome_principle[year]))
        if principles[principle_index] =="utilitarian":
            tuples = [(principles[principle_index],2005+index_year),]*(175*10**3)
        else:
            tuples = [(principles[principle_index],2005+index_year),]*14*10**4
        index_1 = pd.MultiIndex.from_tuples(tuples, names=['principle', 'year'])
        year_df = pd.DataFrame(data =outcome_principle[year], columns= region_pop_gr.index,index=index_1)

        index_year = index_year +10
        POP_total = pd.concat([POP_total,year_df],axis=0)
    principle_index = principle_index + 1


In [ ]:
POP_total

In [ ]:
POP_total.to_csv("short_term_POP_per_time_period.csv")

### get per quintile outputs and save in CSV

In [ ]:
#save CPC as csv
pop_keys = outcome_name_supplementary_quintile[0:11]

quintile = 0
tupple = []
for i in range(0,10):
    run = i
    if quintile ==5:
        quintile = 0
    tupple_to_append = ('egal',year,run,quintile)
    tupple.append(tupple_to_append)
    quintile = quintile + 1
    
index = pd.MultiIndex.from_tuples(tupple,names=['principle', 'year','run','quintile'])
CPC_total = pd.DataFrame(columns= region_pop_gr.index,index = index)

principles = ['egalitarian','egalitarian','sufficitarian','sufficitarian','prioritarian','prioritarian',
              'utilitarian']
outcomes = [outcomes_ega_4_8,outcomes_ega_8_8,outcomes_suf_4_8,
            outcomes_suf_8_8,outcomes_prio_4_8,outcomes_prio_8_8,
            outcomes_util]

index_year = 0
principle_index =0 

for principle in principles:
    outcome = outcomes[principle_index]
    for year in pop_keys:
        #set up tupple 
        quintile = 0
        tupple = []

        for i in range(0,len(outcome[year])*5):
            run = i
            if quintile ==5:
                quintile = 0
                
            tupple_to_append = (principle,year,run,quintile)
            tupple.append(tupple_to_append)
            quintile = quintile + 1
        #build index for dataframe to store results per saved timestep 
        index = pd.MultiIndex.from_tuples(tupple,names=['principle', 'year','run','quintile'])

        CPC_df = pd.DataFrame(data = outcome[year].reshape(-1,outcome[year].shape[-1]),columns= region_pop_gr.index,index =index)

        CPC_total = pd.concat([CPC_total,CPC_df],axis=0)
    principle_index = principle_index + 1


In [ ]:
CPC_total

In [ ]:
CPC_total.to_csv("short_term_CPC_before_damages_per_time_period.csv")

In [ ]:
#save CPC as csv
pop_keys = outcome_name_supplementary_quintile[11:22]

quintile = 0
tupple = []
for i in range(0,10):
    run = i
    if quintile ==5:
        quintile = 0
    tupple_to_append = ('egal',year,run,quintile)
    tupple.append(tupple_to_append)
    quintile = quintile + 1
    
index = pd.MultiIndex.from_tuples(tupple,names=['principle', 'year','run','quintile'])
CPC_total_after = pd.DataFrame(columns= region_pop_gr.index,index = index)

principles = ['egalitarian','egalitarian','sufficitarian','sufficitarian','prioritarian','prioritarian',
              'utilitarian']
outcomes = [outcomes_ega_4_8,outcomes_ega_8_8,outcomes_suf_4_8,
            outcomes_suf_8_8,outcomes_prio_4_8,outcomes_prio_8_8,
            outcomes_util]

index_year = 0
principle_index =0 

for principle in principles:
    outcome = outcomes[principle_index]
    for year in pop_keys:
        #set up tupple 
        quintile = 0
        tupple = []

        for i in range(0,len(outcome[year])*5):
            run = i
            if quintile ==5:
                quintile = 0
                
            tupple_to_append = (principle,year,run,quintile)
            tupple.append(tupple_to_append)
            quintile = quintile + 1

        #build index for dataframe to store results per saved timestep 
        index = pd.MultiIndex.from_tuples(tupple,names=['principle', 'year','run','quintile'])

        CPC_df = pd.DataFrame(data = outcome[year].reshape(-1,outcome[year].shape[-1]),columns= region_pop_gr.index,index =index)

        CPC_total_after = pd.concat([CPC_total_after,CPC_df],axis=0)
    principle_index = principle_index + 1


In [ ]:
CPC_total_after.to_csv("short_term_CPC_after_damages_per_time_period.csv")

In [ ]:
CPC_total_after